![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVino_in_Spark_NLP_MPNetForSequenceClassification.ipynb)

# Import OpenVINO MPNetForSequenceClassification models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting MPNetForSequenceClassification models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for MPNetForSequenceClassification from MPNetForSequenceClassification   and they have to be in `Text Classification` category.

## Export and Save HuggingFace model

- Let's install `transformers` package with the `openvino` extension and it's dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.52.4`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.
- Additionally, we need to install `setfit` to load the model components.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.52.4 optimum openvino setfit

We'll use [rodekruis/sml-ukr-message-classifier](https://huggingface.co/rodekruis/sml-ukr-message-classifier). As this is not a pure `transformers` model, we need to export the modules separately and combine them.

In [2]:
from setfit import SetFitModel
from transformers import AutoTokenizer

MODEL_NAME = "rodekruis/sml-ukr-message-classifier"
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

model = SetFitModel.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/179k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Exporting the Tokenizer

In [3]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, export=True)
tokenizer.save_pretrained(ONNX_MODEL)

('onnx_models/rodekruis/sml-ukr-message-classifier/tokenizer_config.json',
 'onnx_models/rodekruis/sml-ukr-message-classifier/special_tokens_map.json',
 'onnx_models/rodekruis/sml-ukr-message-classifier/vocab.txt',
 'onnx_models/rodekruis/sml-ukr-message-classifier/added_tokens.json',
 'onnx_models/rodekruis/sml-ukr-message-classifier/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {ONNX_MODEL}

total 932
-rw-r--r-- 1 root root    964 Jun 24 12:59 special_tokens_map.json
-rw-r--r-- 1 root root   1632 Jun 24 12:59 tokenizer_config.json
-rw-r--r-- 1 root root 710932 Jun 24 12:59 tokenizer.json
-rw-r--r-- 1 root root 231536 Jun 24 12:59 vocab.txt


- As you can see, we need to move `vocab.txt` to assets folder which Spark NLP will look for
- We also need `labels`. These are not contained in the model itself and we will have to fetch them manually. We will save this inside `labels.txt`

In [5]:
!mkdir -p {ONNX_MODEL}/assets && mv {ONNX_MODEL}/vocab.txt {ONNX_MODEL}/assets/

In [6]:
!wget https://huggingface.co/{MODEL_NAME}/raw/main/label_dict.json

import json

with open("label_dict.json") as f:
    labels = json.load(f)

labels = [value for key, value in sorted(labels.items(), key=lambda x: int(x[0]))]

with open(f"{ONNX_MODEL}/assets/labels.txt", "w") as f:
    f.write("\n".join(labels))

--2025-06-24 12:59:57--  https://huggingface.co/rodekruis/sml-ukr-message-classifier/raw/main/label_dict.json
Resolving huggingface.co (huggingface.co)... 65.8.243.90, 65.8.243.16, 65.8.243.46, ...
Connecting to huggingface.co (huggingface.co)|65.8.243.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 589 [text/plain]
Saving to: ‘label_dict.json’


label_dict.json       0%[                    ]       0  --.-KB/s               
label_dict.json     100%[===================>]     589  --.-KB/s    in 0s      

2025-06-24 12:59:57 (272 MB/s) - ‘label_dict.json’ saved [589/589]



In [7]:
ls -l {ONNX_MODEL}/assets

total 232
-rw-r--r-- 1 root root    337 Jun 24 12:59 labels.txt
-rw-r--r-- 1 root root 231536 Jun 24 12:59 vocab.txt


Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

## Combining and exporting the SetFit Modules

The `SetFitModel` is composed of these components, we need to export:

1. MPNet Embeddings Model
2. Pooling Module
3. Normalization Module
4. Prediction Module

We first create a custom torch module, to export it into a single ONNX graph.

In [8]:
import torch
from torch import nn

class SentencePredictor(nn.Module):
    """
    A PyTorch model wrapper for sentence-level predictions.
    Uses pre-trained embeddings, pooling, and normalization layers from the given model.
    """
    def __init__(self, model):
        super().__init__()

        # Load model coefficients and intercept from the trained model head
        self.coeffs = torch.Tensor(model.model_head.coef_)
        self.intercept = torch.Tensor(model.model_head.intercept_)

        # Extract model body components: embeddings, pooling, and normalization
        self.embeddings, self.pooling, self.normalize = model.model_body

    def predict(self, normed_embeddings):
        """
        Compute logits using normalized embeddings.
        """
        logits = normed_embeddings @ self.coeffs.T + self.intercept
        return logits

    def forward(self, input_ids, attention_mask):
        """
        Forward pass through the model:
        - Generate embeddings
        - Apply pooling
        - Normalize the pooled embeddings
        - Compute logits
        """
        input_data = {"input_ids": input_ids, "attention_mask": attention_mask}
        embeddings_out = self.embeddings(input_data)
        pooling_out = self.pooling(embeddings_out)
        normalize_out = self.normalize(pooling_out)

        logits = self.predict(normalize_out["sentence_embedding"])
        return {"logits": logits}

# Instantiate the SentencePredictor model with the given pre-trained model
sp = SentencePredictor(model)

# Tokenize input sentences using the model's tokenizer
input_data = model.model_body.tokenize([
    "I loved the Spiderman movie!",
    "Pineapple on pizza is the worst 🤮"
])

# Export the model to ONNX format for optimized inference
torch.onnx.export(
    sp,
    args=input_data,
    f=f"{ONNX_MODEL}/model.onnx",
    input_names=["input_ids", "attention_mask"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "token_length"},
        "attention_mask": {0: "batch_size", 1: "token_length"},
        "logits": {0: "batch_size"},
    }
)

Now we have the model and all necessary files to import it into Spark NLP!

In [9]:
!ls -lR {ONNX_MODEL}

onnx_models/rodekruis/sml-ukr-message-classifier:
total 426468
drwxr-xr-x 2 root root      4096 Jun 24 12:59 assets
-rw-r--r-- 1 root root 435970222 Jun 24 13:00 model.onnx
-rw-r--r-- 1 root root       964 Jun 24 12:59 special_tokens_map.json
-rw-r--r-- 1 root root      1632 Jun 24 12:59 tokenizer_config.json
-rw-r--r-- 1 root root    710932 Jun 24 12:59 tokenizer.json

onnx_models/rodekruis/sml-ukr-message-classifier/assets:
total 232
-rw-r--r-- 1 root root    337 Jun 24 12:59 labels.txt
-rw-r--r-- 1 root root 231536 Jun 24 12:59 vocab.txt


Now let's convert and save the ONNX model in OpenVINO format

In [10]:
import openvino as ov

ov.save_model(ov.convert_model(f"{ONNX_MODEL}/model.onnx"), "openvino_model.xml")

!rm -rf {ONNX_MODEL}/model.onnx
!mv /content/openvino_model.* {ONNX_MODEL}

## Import and Save MPNetForSequenceClassification in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [11]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 41.7 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [12]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 5.5.3
Apache Spark version: 3.5.4


- Let's use `loadSavedModel` functon in `MPNetForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `MPNetForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [13]:
from sparknlp.annotator import MPNetForSequenceClassification

sequenceClassifier = (
    MPNetForSequenceClassification.loadSavedModel(ONNX_MODEL, spark)
    .setInputCols(["document", "token"])
    .setOutputCol("label")
)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [14]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [15]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your AlbertForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [16]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 213320
drwxr-xr-x 4 root root      4096 Jun 24 13:03 fields
drwxr-xr-x 2 root root      4096 Jun 24 13:03 metadata
-rw-r--r-- 1 root root 218431440 Jun 24 13:03 mpnet_classification_openvino


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForSequenceClassification model 😊

In [17]:
sequenceClassifier_loaded = (
    MPNetForSequenceClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))
    .setInputCols(["document", "token"])
    .setOutputCol("label")
)

You can see what labels were used to train this model via `getClasses` function:

In [18]:
sequenceClassifier_loaded.getClasses()

['GOODS/SERVICES',
 'EDUCATION',
 'SHELTER',
 'OTHER PROGRAMS/NGOS',
 'RC PROGRAM INFO',
 'CVA REGISTRATION',
 'CAR',
 'ARMY',
 'PSS & RFL',
 'CVA PAYMENT',
 'CHILDREN',
 'CONNECTIVITY',
 'CVA INCLUSION',
 'FOOD',
 'HEALTH',
 'TRANSLATION/LANGUAGE',
 'LEGAL',
 'CVA PROGRAM INFO',
 'PETS',
 'MONEY/BANKING',
 'WORK/JOBS',
 'RC CONNECT WITH RED CROSS',
 'PARCEL',
 'TRANSPORT/MOVEMENT',
 'NFI',
 'ANOMALY',
 'RC PMER/NEW PROGRAMS',
 'WASH',
 'SENTIMENT']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [19]:
from sparknlp.annotator import Tokenizer
from sparknlp.base import DocumentAssembler
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier_loaded
])

example = spark.createDataFrame([
    ["I love driving my car."],
    ["The next bus will arrive in 20 minutes."],
    ["Pineapple on pizza is the worst 🤮"]
]).toDF("text")

model = pipeline.fit(example)
result = model.transform(example)

result.select("text", "label.result").show()

+--------------------+--------------------+
|                text|              result|
+--------------------+--------------------+
|I love driving my...|               [CAR]|
|The next bus will...|[TRANSPORT/MOVEMENT]|
|Pineapple on pizz...|              [FOOD]|
+--------------------+--------------------+



That's it! You can now go wild and use hundreds of `MPNetForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀
